<a href="https://colab.research.google.com/github/Lastdragonlord0403/FakeNewsPrediction/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[source](https://youtu.be/nacLBdyG6jE)

WORK FLOW
1. Collect news labeled data
2. Data processing
3. Train test split
4. Feed training data: use logistic regression model (binanry classificatin: classify the reslt into 2 types: real/fake)
5. train logistic regression model: do some evalations on this model
-> find the accuracy score using the test data
6. input news data -> predict

About the Dataset:
contain 5 columns

1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News





Importing the Dependencies

In [1]:
# import library

# useful for making numpy arrays
import numpy as np

# read file, create data frames, store the data in the data frame
import pandas as pd

# searching the text in document
import re

# nltk: natural language toolkit
# corpus: the body of that particular text is important content of the text
# stop wors: remove those word which doesn't add much value to a paragraph vertex
from nltk.corpus import stopwords

# function stemming: takes a word & removes the prefix & suffix of the word
# return: the root word of it
from nltk.stem.porter import PorterStemmer

# function: TfidVectorizer: convert the text into feature vectors which are nothing but number
from sklearn.feature_extraction.text import TfidfVectorizer

# train test split
# split our data set into training data & test data
from sklearn.model_selection import train_test_split

# logistic model
from sklearn.linear_model import LogisticRegression

# the accurarcy score
from sklearn.metrics import accuracy_score

In [2]:
# download stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# printing the stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# input the dataset to a pandas DataFrame
news_dataset = pd.read_csv('/content/sample_data/train.csv')

In [7]:
# check the number of rows & column in the data set
news_dataset.shape

(20800, 5)

In [8]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
# counting the number of missing values in the dataset
# null string or empty string?
news_dataset.isnull().sum()

# in this case, the missing value doesn't affect too much
# we have large dataset, the full data still enough to train
# can replace missing ones

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [11]:
# data has title, authour
# combine them together, this is the data we re going to use for predictions
# not use text bcz these text is so large
# good accuracy score, good performance

# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [12]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [13]:
# separating the data & label
# remove a column: axis = 1
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

# remove column 'label' out of data frame
# store rest data in one variable
# store label alone in one variable

In [14]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming:

Stemming is the process of reducing a word to its Root word

example:
actor, actress, acting --> act

This is improtant to reduce the words as much as possible to have better performance on our model

function: vectorizing: conert these words to respective feauture vectors (numerical data)
-> feed machine learning model

In [15]:
port_stem = PorterStemmer()

In [16]:
# this is not an inbuilt function so we have to create
def stemming(content):

  # sub: subtitues certain values
  # ^a: exclusion
  # remove all thoses things from content except for alphabet and words
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)

  # convert all the alphabets to lowercase letters
    stemmed_content = stemmed_content.lower()

  # split content respective to list
    stemmed_content = stemmed_content.split()

  # stemming each word & remove stopwords
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]

  # joining all the words
    stemmed_content = ' '.join(stemmed_content)

  # return processed text
    return stemmed_content

In [17]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [18]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [19]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [20]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [21]:
print(Y)

[1 0 1 ... 0 1 1]


In [22]:
Y.shape

(20800,)

In [23]:
# value of content still in textual form
# computer can not understand text
# need to convert the textual data to numerical data that computer can understand

# tf: term frequency
# assign particular numerical value to a word
# idf: transfer inverse document frequency

vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [24]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splitting the dataset to training & test data

In [25]:
# 80%: train, 20%: test
# label of X data will be stored in Y data
# stratifies=Y: thre will be a similar propotion as it was in the original data set
# if don't have stratisfies: fake & real news won't be segregated in an equal proprtion
# random=2: reproduce a particular code
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

Training the Model: Logistic Regression

In [26]:
model = LogisticRegression()

In [27]:
# fit func: train model
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [28]:
# accuracy score on the training data

# want model to make prdiction on extreme data
# all thoese models the labels that are predicted and stored in X_test_prediction
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [29]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9865985576923076


accuracy score on training data is not that much important but on test data is important. Bcz our model is trained on training data but it hasn't seen what is the test data so the acuracy score on test data will tell us how good our model is performing.

In [30]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [31]:
print('Accuracy score of the test data : ', test_data_accuracy)

# around 99%: very good accuracy score

Accuracy score of the test data :  0.9790865384615385


Making a Predictive System

In [32]:
# when we have a new value our model can predict label & tell it's real or fake
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [33]:
# fixed value
print(Y_test[3])

0
